In [1]:
import autograd.numpy as np
import matplotlib.pyplot as plt
from autograd import grad, elementwise_grad, hessian, jacobian

import sys
sys.path.insert(0, "../../project_2/src")
from SGD import minibatch

In [2]:
# Inspired by mortens example code in week 43

def activation_function(z):
    return np.maximum(0, z)

def activation_out(z):
    return z

"""
network_shape = [inputs, w1, w2, ..., wL, outputs]

W = wwwwb
    wwwwb
"""
def initialize_params(network_shape):
    P = []
    for i in range(1, len(network_shape)):
        k = network_shape[i-1]
        j = network_shape[i]
        P.append(np.random.randn(j, k) * np.sqrt(2) / np.sqrt(k))
        P[i-1] = np.concatenate((P[i-1], np.zeros(j).reshape(-1,1)), axis=1)

    return P

def Network(t, P):
    # Assumes the input t to be a single-element 1d-array, returns a vector
    a = t.reshape(-1,1)
    for P_i in P:
        #a = np.concatenate((a, np.ones(np.size(a, 0)).reshape(-1, 1)), axis=1) 
        a = np.concatenate((a, np.ones((1,1))), axis=1) 
        z = np.matmul(a, np.transpose(P_i))
        a = activation_function(z)
    
    return activation_out(z) #check the returned shape

def optimize(t, P, A, x_0, N_minibatches, learning_rate, n_epochs):
    
    assert N_minibatches <= np.size(t, 0)

    cost_func_grad = grad(costfunction, 1) # Check which grad-call is correct.
    
    for epoch in range(n_epochs):
        mb = minibatch(t, N_minibatches)
        for i in range(N_minibatches):
            t_mb = t[mb[i]]
            M = np.size(t_mb, 0)
            # compute gradients of weights
            cost_grad = cost_func_grad(t_mb, P, A, x_0)
            for l in range(len(P)):
                P[l] -= learning_rate * cost_grad[l]
    return P

def g_trial(t, P, x_0):
    #assumes the input t to be a single-element 1d-array
    
    return np.exp(t)*x_0 + (1-exp(t))*Network(t,P)

def costfunction(t, P, A, x_0):
    
    cost = 0
    g_grad = grad(g_trial,0) # Check that this is the correct grad-call
    
    for time in t:
        d_dt = g_grad(time,P,x_0)
        x_t = g_trial(t,P,x_0)
        right_side = -x_t + () # -x + [x.T @ x @ A + (1 - x.T @ A @ x) @ I] @ x where x is x_t is a column vector
        cost += np.sum((d_dt - right_side)**2) / np.size(x_t) #beware the increment operator may not play nice with
                                                              # Autograd
        

    return cost / np.size(t)

In [3]:
t = np.linspace(0,1,10)
Q = np.random.randn(6,6)
A = (np.transpose(Q) + Q) /2 #standard trick for making symmetric real
x_0 = np.random.randn(6) #initial guess for eigenvector.

network_shape = [1, 10, np.size(x_0)] #output must match eigenvector length
P = initialize_params(network_shape)

In [4]:
P = optimize(t, P, A, x_0,10, 0.002, 10)
#y = Network(X[0, :].reshape(1, 2), P)